In [35]:
import tensorflow as tf

In [36]:
import pandas as pd

In [37]:
import numpy as np

In [38]:
import itertools

In [39]:
import os

In [40]:
import tempfile

In [41]:
import pickle

In [42]:
from sklearn import datasets

In [43]:
from IPython.display import clear_output

In [44]:
training_set_n = pd.read_csv('/Users/wisdom/Downloads/boston_train/boston_train.csv').values

In [45]:
test_set_n = pd.read_csv('/Users/wisdom/Downloads/boston_train/boston_test.csv').values

In [120]:
prediction_set_n = pd.read_csv('/Users/wisdom/Downloads/boston_train/boston_predict.csv').values

In [48]:
def prepare_data(df):
    X_train = df[:, :-3]
    y_train = df[:,-3]    
    return X_train, y_train	

In [49]:
X_train, y_train = prepare_data(training_set_n)

In [50]:
X_test, y_test = prepare_data(test_set_n)

In [127]:
X_predict = prediction_set_n[:, :-2]

In [128]:
print(X_predict)

[[3.35900e-02 7.50000e+01 2.95000e+00 4.28000e-01 7.02400e+00 1.58000e+01
  5.40110e+00 2.52000e+02 1.83000e+01         nan         nan]
 [5.09017e+00 0.00000e+00 1.81000e+01 7.13000e-01 6.29700e+00 9.18000e+01
  2.36820e+00 6.66000e+02 2.02000e+01         nan         nan]
 [1.26500e-01 2.50000e+01 5.13000e+00 4.53000e-01 6.76200e+00 4.34000e+01
  7.98090e+00 2.84000e+02 1.97000e+01         nan         nan]
 [5.51500e-02 3.30000e+01 2.18000e+00 4.72000e-01 7.23600e+00 4.11000e+01
  4.02200e+00 2.22000e+02 1.84000e+01         nan         nan]
 [8.15174e+00 0.00000e+00 1.81000e+01 7.00000e-01 5.39000e+00 9.89000e+01
  1.72810e+00 6.66000e+02 2.02000e+01         nan         nan]
 [2.45220e-01 0.00000e+00 9.90000e+00 5.44000e-01 5.78200e+00 7.17000e+01
  4.03170e+00 3.04000e+02 1.84000e+01         nan         nan]]


In [54]:
print(X_train.shape, y_train.shape, X_predict.shape)

(400, 9) (400,) (6, 9)


In [55]:
feature_columns = [tf.feature_column.numeric_column('x', shape=X_train.shape[1:])]

In [56]:
print(feature_columns)

[NumericColumn(key='x', shape=(9,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


In [57]:
estimator = tf.estimator.LinearRegressor(feature_columns=feature_columns, model_dir="train")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'train_n', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fe9c603bd90>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [58]:
def get_input_fn(data_df, label_df, num_epochs=None, batch_size=128, shuffle=True, steps=1000):
    def input_fn():
        ds = tf.data.Dataset.from_tensor_slices(({"x": data_df}, label_df)) if label_df is not None else tf.data.Dataset.from_tensor_slices(({"x": data_df}))
        if shuffle:
            ds = ds.shuffle(steps)
        ds = ds.batch(batch_size)
        return ds
    return input_fn     

In [59]:
train = estimator.train(input_fn=get_input_fn(X_train, y_train, shuffle=False))


INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from train_n/model.ckpt-186
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 186 into train_n/model.ckpt.
INFO:tensorflow:loss = 99.647705, step = 186
INFO:tensorflow:Saving checkpoints for 190 into train_n/model.ckpt.
INFO:tensorflow:Loss for 

In [60]:
ev = estimator.evaluate(input_fn=get_input_fn(X_test, y_test, shuffle=False))
print(ev)

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-01-03T19:57:26Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from train_n/model.ckpt-190
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2021-01-03-19:57:27
INFO:tensorflow:Saving dict for global step 190: average_loss = 50.994217, global_step = 190, label/mean = 22.08, loss = 50.994217,

In [61]:
loss_score = ev["loss"]
print("Loss: {0:f}".format(loss_score))

Loss: 50.994217


In [62]:
y = estimator.predict(input_fn=get_input_fn(X_predict, None, shuffle=False))

In [63]:
pred_dicts = list(estimator.predict(input_fn=get_input_fn(X_predict, None, shuffle=False)))
clear_output()
for pred in pred_dicts:
    print(pred["predictions"][0])

30.937294
21.649946
27.398323
25.803082
20.206217
21.798332


In [64]:
pred_dicts = list(y)
for pred in pred_dicts:
    print(pred["predictions"][0])

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from train_n/model.ckpt-190
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
30.937294
21.649946
27.398323
25.803082
20.206217
21.798332


In [65]:
tmpdir = tempfile.mkdtemp()
estimator_base_path = os.path.join(tmpdir, 'from_estimator')
feature_spec = tf.feature_column.make_parse_example_spec(feature_columns);
export_input_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(feature_spec);
estimator_path = estimator.export_saved_model(estimator_base_path, export_input_fn)
imported = tf.saved_model.load(estimator_path)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: ['serving_default', 'regression']
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['predict']
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:Restoring parameters from train_n/model.ckpt-190
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: /var/folders/6_/8_x7k80s52vcfppnmvb31wxr0000gn/T/tmpdyyytqr1/from_estimator/temp-b'1609703847'/saved_model.pb


In [118]:
def predict(x):
    examples = []
    for row in x:
        print(row)
        feature = {}
        feature["x"] = tf.train.Feature(float_list=tf.train.FloatList(value=row))
        example = tf.train.Example(
            features=tf.train.Features(
                feature=feature
            )
        )
        examples.append(example.SerializeToString())
    return imported.signatures["predict"](
    examples=tf.constant(examples))

In [194]:
print(predict(X_predict))

{'predictions': <tf.Tensor: id=7303, shape=(6, 1), dtype=float32, numpy=
array([[30.917381],
       [21.590858],
       [27.513954],
       [25.884512],
       [20.11757 ],
       [21.866253]], dtype=float32)>}
